In [1]:
from __future__ import print_function

from scipy.misc import imsave
import numpy as np
import time
import matplotlib.pyplot as plt

from keras import backend as K
from keras import activations
from keras.models import load_model
#from dataset_models.sdo.aia.layers import LogWhiten
from layers import LogWhiten

#from vis.utils import utils
import utils

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


Using TensorFlow backend.


In [2]:

def relu_advanced(x):
    return K.relu(x, alpha=0.01)    

# dimensions of the generated pictures for each filter.

img_width = 1024
img_height = 1024

#Reading model

#network_model_path = '/home/solardynamo/solar-forecast/network_models/xray_flux_forecast/fdl4/trained_models/good/20170814-020747/epochs/weights.99-1489.53.hdf5'
#network_model_path = 'weights.99-1489.53.hdf5'
#network_model_path = 'weights.34-2472.80.hdf5'
#filename = '2nd_Dreamer.weights.128-1526.14'
#filename = '1st_Dreamer_weights.99-1489.53'
#filename = '3rd_Dreamer_weights.38-482.85'
#network_model_path = filename +'.hdf5'
#custom_objects = {"LogWhiten": LogWhiten}


#filename = 'FN7_Leak_weights.167-331.71'
#filename = 'FN6_weights.197-198.27'

#filename = 'FN9_weights.175-255.91'
filename = 'FN8_weights.90-556.06'


network_model_path = filename +'.hdf5'
custom_objects = {"LogWhiten": LogWhiten, "relu_advanced":relu_advanced}



model = load_model(network_model_path, custom_objects=custom_objects)


model.summary()


# the name of the layer we want to visualize
# (see model definition at keras/applications/vgg16.py)

layer_name = 'conv2d_6'

layer_idx = utils.find_layer_idx(model, layer_name)

model.layers[layer_idx].activation = activations.linear
model.save('tmp.hdf5')
model = load_model('tmp.hdf5', custom_objects=custom_objects)


# utility function to convert a tensor into a valid image
def deprocess_image(x):
    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + 1e-5)
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    #x = x.transpose((1, 2, 0))
    x = np.clip(x, 0, 255).astype('uint8')
    return x


# utility function to normalize a tensor by its L2 norm
def normalize(x):

    return x / (K.sqrt(K.mean(K.square(x))) + 1e-5)



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1024, 1024, 8)     0         
_________________________________________________________________
log_whiten_1 (LogWhiten)     (None, 1024, 1024, 8)     0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 1024, 1024, 16)    8192      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1024, 1024, 16)    16384     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 128, 128, 16)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 128, 128, 16)      16384     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 128, 128, 16)      16384     
__________

In [3]:
#this is the placeholder for the input images
input_img = model.input

# get the symbolic outputs of each "key" layer (we gave them unique names).
layer_dict = dict([(layer.name, layer) for layer in model.layers[1:]])

In [4]:
filters = []

output_folder1 = filename + '/' + layer_name + '/1600_304_171'
if not os.path.exists(output_folder1):
    os.makedirs(output_folder1)

output_folder2 = filename + '/' + layer_name + '/171_193_211'
if not os.path.exists(output_folder2):
    os.makedirs(output_folder2)
    
output_folder3 = filename + '/' + layer_name + '/335_94_131'
if not os.path.exists(output_folder3):
    os.makedirs(output_folder3)
    
    
for filter_index in range(0, 16):
    
    plt.close("all")

    # we only scan through the first 200 filters,
    # but there are actually 512 of them

    print('Processing filter %d' % filter_index)
    start_time = time.time()

    # we build a loss function that maximizes the activation
    # of the nth filter of the layer considered

    layer_output = layer_dict[layer_name].output

    if K.image_data_format() == 'channels_first':
        loss = K.mean(layer_output[:, filter_index, :, :])
    else:
        loss = K.mean(layer_output[:, :, :, filter_index])

        
    # we compute the gradient of the input picture wrt this loss
    grads = K.gradients(loss, input_img)[0]
   
    # normalization trick: we normalize the gradient
    grads = normalize(grads)

    # this function returns the loss and grads given the input picture
    iterate = K.function([input_img], [loss, grads])

    # step size for gradient ascent
    step = 1000
    

    # we start from a gray image with some random noise

    if K.image_data_format() == 'channels_first':
        input_img_data = np.random.randn(1, 8, img_width, img_height)
    else:
        input_img_data = np.random.randn(1, img_width, img_height, 8)

    input_img_data = input_img_data*20+20
    input_img_data[input_img_data<=0] = 0;


    # we run gradient ascent for 20 steps
    for i in range(150):
        loss_value, grads_value = iterate([input_img_data])
        input_img_data += grads_value * step

        print('Current loss value:', loss_value)
        if loss_value == 0.:
            break
        


    afilter = deprocess_image(input_img_data[0])
    
    print(afilter.shape)

    #Assemble filters for composite 1

    #1600
    r = afilter[:,:,7]
    #304
    g = afilter[:,:,5]
    #171
    b = afilter[:,:,2]

    rgb1 = np.dstack((r,g,b))

    fig = plt.figure(frameon=False)
    fig.set_size_inches(10, 10)
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    ax.imshow(rgb1)
    plt.axis('off')
    plt.tick_params(axis='both', left='off', top='off', right='off', bottom='off', labelleft='off', labeltop='off', labelright='off', labelbottom='off')
    fig.savefig(output_folder1 + '/' + layer_name + '_' + str(filter_index) + '.png', bbox_inches='tight', dpi = 103, pad_inches=0)


    #Assemble filters for composite 2

    #171
    r = afilter[:,:,2]
    #193
    g = afilter[:,:,3]
    #211
    b = afilter[:,:,4]

    rgb1 = np.dstack((r,g,b))

    fig = plt.figure(frameon=False)
    fig.set_size_inches(10, 10)
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    ax.imshow(rgb1)
    plt.axis('off')
    plt.tick_params(axis='both', left='off', top='off', right='off', bottom='off', labelleft='off', labeltop='off', labelright='off', labelbottom='off')
    fig.savefig(output_folder2 + '/' + layer_name + '_' + str(filter_index) + '.png', bbox_inches='tight', dpi = 103, pad_inches=0)


    #Assemble filters for composite 3

    #335
    r = afilter[:,:,6]
    #94
    g = afilter[:,:,0]
    #131
    b = afilter[:,:,1]

    rgb1 = np.dstack((r,g,b))


    fig = plt.figure(frameon=False)
    fig.set_size_inches(10, 10)
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    ax.imshow(rgb1)
    plt.axis('off')
    plt.tick_params(axis='both', left='off', top='off', right='off', bottom='off', labelleft='off', labeltop='off', labelright='off', labelbottom='off')
    fig.savefig(output_folder3 + '/' + layer_name + '_' + str(filter_index) + '.png', bbox_inches='tight', dpi = 103, pad_inches=0)
        

    end_time = time.time()
    print('Filter %d processed in %ds' % (filter_index, end_time - start_time))

Processing filter 0
Current loss value: -92.6123
Current loss value: -193.281
Current loss value: -50.4317
Current loss value: 49.3489
Current loss value: 87.9514
Current loss value: 109.728
Current loss value: 127.01
Current loss value: 137.128
Current loss value: 143.739
Current loss value: 148.223
Current loss value: 152.977
Current loss value: 157.524
Current loss value: 151.571
Current loss value: 164.474
Current loss value: 169.445
Current loss value: 168.496
Current loss value: 173.964
Current loss value: 178.033
Current loss value: 181.377
Current loss value: 183.894
Current loss value: 186.818
Current loss value: 185.811
Current loss value: 191.686
Current loss value: 195.561
Current loss value: 198.118
Current loss value: 199.767
Current loss value: 202.038
Current loss value: 204.943
Current loss value: 208.74
Current loss value: 212.521
Current loss value: 214.738
Current loss value: 219.179
Current loss value: 223.174
Current loss value: 228.074
Current loss value: 229.134

Current loss value: 555.188
Current loss value: 557.01
Current loss value: 559.711
Current loss value: 561.165
Current loss value: 564.099
Current loss value: 566.444
Current loss value: 567.766
Current loss value: 569.962
Current loss value: 571.161
(1024, 1024, 8)
Filter 1 processed in 213s
Processing filter 2
Current loss value: -90.4661
Current loss value: -179.331
Current loss value: -33.7085
Current loss value: 32.7564
Current loss value: 68.649
Current loss value: 92.4088
Current loss value: 109.416
Current loss value: 117.216
Current loss value: 127.163
Current loss value: 133.375
Current loss value: 138.89
Current loss value: 142.967
Current loss value: 148.203
Current loss value: 152.036
Current loss value: 155.022
Current loss value: 160.396
Current loss value: 163.458
Current loss value: 166.026
Current loss value: 169.965
Current loss value: 172.745
Current loss value: 176.164
Current loss value: 179.128
Current loss value: 182.052
Current loss value: 185.468
Current loss 

Current loss value: 505.194
Current loss value: 505.886
Current loss value: 508.95
Current loss value: 510.454
Current loss value: 510.609
Current loss value: 512.767
Current loss value: 514.784
Current loss value: 516.783
Current loss value: 518.796
Current loss value: 521.112
Current loss value: 522.721
Current loss value: 523.784
Current loss value: 525.374
Current loss value: 526.777
Current loss value: 529.133
Current loss value: 530.957
Current loss value: 532.561
Current loss value: 533.946
Current loss value: 535.122
Current loss value: 537.08
(1024, 1024, 8)
Filter 3 processed in 213s
Processing filter 4
Current loss value: -113.484
Current loss value: -272.911
Current loss value: 5.63397
Current loss value: 81.8382
Current loss value: 127.08
Current loss value: 155.523
Current loss value: 175.438
Current loss value: 188.374
Current loss value: 202.025
Current loss value: 209.516
Current loss value: 217.962
Current loss value: 225.63
Current loss value: 232.187
Current loss va

Current loss value: 195.64
Current loss value: 196.288
Current loss value: 197.845
Current loss value: 198.686
Current loss value: 199.813
Current loss value: 199.993
Current loss value: 201.038
Current loss value: 201.418
Current loss value: 202.838
Current loss value: 203.522
Current loss value: 204.406
Current loss value: 205.924
Current loss value: 206.447
Current loss value: 206.947
Current loss value: 208.141
Current loss value: 209.279
Current loss value: 209.955
Current loss value: 211.169
Current loss value: 212.134
Current loss value: 212.546
Current loss value: 211.735
Current loss value: 214.23
Current loss value: 215.211
Current loss value: 216.699
Current loss value: 218.029
Current loss value: 218.196
Current loss value: 219.694
Current loss value: 221.51
Current loss value: 222.279
Current loss value: 219.074
(1024, 1024, 8)
Filter 5 processed in 213s
Processing filter 6
Current loss value: -106.68
Current loss value: -177.343
Current loss value: -63.7973
Current loss v

Current loss value: 496.194
Current loss value: 500.139
Current loss value: 503.919
Current loss value: 506.527
Current loss value: 510.537
Current loss value: 513.51
Current loss value: 517.268
Current loss value: 521.043
Current loss value: 524.644
Current loss value: 528.58
Current loss value: 530.957
Current loss value: 535.072
Current loss value: 538.046
Current loss value: 542.334
Current loss value: 545.505
Current loss value: 548.577
Current loss value: 551.174
Current loss value: 553.974
Current loss value: 557.249
Current loss value: 563.078
Current loss value: 566.609
Current loss value: 569.755
Current loss value: 572.651
Current loss value: 577.397
Current loss value: 580.838
Current loss value: 583.788
Current loss value: 586.618
Current loss value: 590.119
Current loss value: 594.004
Current loss value: 597.282
Current loss value: 600.677
Current loss value: 602.99
Current loss value: 606.4
Current loss value: 608.592
Current loss value: 613.234
Current loss value: 616.6

Current loss value: 430.484
Current loss value: 429.769
Current loss value: 432.639
Current loss value: 434.639
Current loss value: 436.463
Current loss value: 438.088
Current loss value: 439.436
Current loss value: 441.393
Current loss value: 443.708
Current loss value: 445.637
Current loss value: 448.374
Current loss value: 449.501
Current loss value: 452.542
Current loss value: 454.572
Current loss value: 454.27
Current loss value: 458.297
Current loss value: 460.524
Current loss value: 463.033
Current loss value: 464.802
Current loss value: 466.846
Current loss value: 468.756
Current loss value: 471.58
Current loss value: 472.852
Current loss value: 474.753
Current loss value: 477.391
Current loss value: 478.499
Current loss value: 480.113
Current loss value: 481.494
Current loss value: 484.412
Current loss value: 486.758
Current loss value: 487.051
Current loss value: 488.617
Current loss value: 489.756
Current loss value: 492.03
Current loss value: 493.278
Current loss value: 494

Current loss value: 142.377
Current loss value: 143.849
Current loss value: 145.597
Current loss value: 146.425
Current loss value: 148.03
Current loss value: 150.114
Current loss value: 152.092
Current loss value: 154.065
Current loss value: 156.45
Current loss value: 157.224
Current loss value: 160.134
Current loss value: 161.414
Current loss value: 163.952
Current loss value: 166.028
Current loss value: 168.167
Current loss value: 169.489
Current loss value: 171.061
Current loss value: 172.475
Current loss value: 174.076
Current loss value: 174.308
Current loss value: 176.255
Current loss value: 178.379
Current loss value: 179.872
Current loss value: 181.278
Current loss value: 182.379
Current loss value: 184.2
Current loss value: 185.644
Current loss value: 186.594
Current loss value: 188.101
Current loss value: 189.457
Current loss value: 190.813
Current loss value: 192.01
Current loss value: 192.8
Current loss value: 194.702
Current loss value: 195.67
Current loss value: 197.126


Current loss value: 573.577
Current loss value: 580.899
Current loss value: 585.914
Current loss value: 592.985
Current loss value: 598.846
Current loss value: 607.581
Current loss value: 612.848
Current loss value: 620.086
Current loss value: 626.431
Current loss value: 632.462
Current loss value: 637.149
Current loss value: 643.393
Current loss value: 649.541
Current loss value: 656.071
Current loss value: 662.65
Current loss value: 668.367
Current loss value: 673.843
Current loss value: 681.542
Current loss value: 688.56
Current loss value: 694.159
Current loss value: 698.897
Current loss value: 703.91
Current loss value: 709.548
Current loss value: 716.489
Current loss value: 721.489
Current loss value: 727.735
Current loss value: 734.05
Current loss value: 739.578
Current loss value: 744.003
Current loss value: 751.471
Current loss value: 756.44
Current loss value: 763.385
Current loss value: 767.905
Current loss value: 774.224
Current loss value: 779.439
Current loss value: 784.6

Current loss value: 313.967
Current loss value: 315.702
Current loss value: 316.008
Current loss value: 320.526
Current loss value: 323.116
Current loss value: 325.15
Current loss value: 327.008
Current loss value: 329.364
Current loss value: 331.199
Current loss value: 331.89
Current loss value: 333.276
Current loss value: 335.619
Current loss value: 338.064
Current loss value: 340.047
Current loss value: 339.764
Current loss value: 344.68
Current loss value: 346.89
Current loss value: 349.214
Current loss value: 351.259
Current loss value: 352.862
Current loss value: 356.047
Current loss value: 356.447
Current loss value: 359.377
Current loss value: 361.613
Current loss value: 364.354
Current loss value: 365.813
Current loss value: 366.935
Current loss value: 367.779
Current loss value: 370.779
Current loss value: 372.977
Current loss value: 371.339
Current loss value: 372.725
Current loss value: 376.206
Current loss value: 377.946
Current loss value: 379.083
Current loss value: 381.

In [5]:
fig = plt.figure(figsize=(10,10))
plt.pcolor(filters[0][0][:,:,0])
plt.show()

IndexError: list index out of range